# Dataset Loading

In [ ]:
#Import important libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas_market_calendars as mcal

In [ ]:
# Load the CRSP and IQ Key Development dataset
CRSP = pd.read_csv('CRSP - Communication Services Dataset.csv')
IQKD = pd.read_csv('IQ Key Development - Communication Services Dataset.csv')

# CRSP Dataset Preprocessing

## Organizing the Company Name and Tickers of CRSP Dataset

In [ ]:
# Columns to be dropped as it's unusable
columns_to_remove_atCRSP = ['PERMNO']
CRSP = CRSP.drop(columns=columns_to_remove_atCRSP)

# Display the modified DataFrame
CRSP.head(3)

In [ ]:
# Group by 'PERMCO' and select the first 'TICKER' and 'COMNAM' for each group
first_values = CRSP.groupby('PERMCO').first()

# Map these first values back to the original DataFrame
CRSP['TICKER'] = CRSP['PERMCO'].map(first_values['TICKER'])
CRSP['COMNAM'] = CRSP['PERMCO'].map(first_values['COMNAM'])

# Now CRSP DataFrame should have standardized 'TICKER' and 'COMNAM' for each 'PERMCO'


In [ ]:
#Summarize of the dataset
def summarize_data(df, name):
    print(f"Summary for {name}:")
    print("Total columns:", df.shape[1])
    print("Total rows:", df.shape[0])
    print("Missing values per column:")
    print(df.isnull().sum())
    print("\n")
summarize_data(CRSP, "CRSP Dataset")

In [ ]:
# Display all rows
pd.set_option('display.max_rows', None)

# Group by 'PERMCO' and aggregate the 'date' column
result = CRSP.groupby('PERMCO')['date'].agg(['min', 'max', 'count'])

# Convert 'min' column to datetime to compare with January 2005
result['min'] = pd.to_datetime(result['min'])

# Filter out rows where 'min' date is earlier than January 2005
filtered_result = result[result['min'] <= '2005-01-03']

# Show the filtered result
print(filtered_result)

# Add the total count of unique PERMCO
total_unique_permco = filtered_result.shape[0]
print(f"Total count of unique PERMCO: {total_unique_permco}")


In [ ]:
# Ensure 'date' is in the correct datetime format
CRSP['date'] = pd.to_datetime(CRSP['date'])

# Convert 'PRC' and 'SHROUT' to numeric to avoid any type issues, dropping NaN values
CRSP.dropna(subset=['PRC', 'SHROUT'], inplace=True)
CRSP['PRC'] = pd.to_numeric(CRSP['PRC'], errors='coerce')
CRSP['SHROUT'] = pd.to_numeric(CRSP['SHROUT'], errors='coerce')

# Group by 'COMNAM' and calculate maximum market cap observed on the fly, and min/max dates
market_cap_result = CRSP.groupby('COMNAM').agg({
    'PRC': lambda x: (x * CRSP.loc[x.index, 'SHROUT']).max(),
    'date': ['min', 'max']
}).reset_index()

# Rename columns to make the DataFrame easier to read
market_cap_result.columns = ['COMNAM', 'max_market_cap', 'min_date', 'max_date']

# Sort the results by market cap in descending order
market_cap_result = market_cap_result.sort_values(by='max_market_cap', ascending=False).reset_index(drop=True)

# Display the result
market_cap_result


In [ ]:
# Count the unique PERMCO values
unique_permco_count = CRSP['PERMCO'].nunique()

# Display the count of unique PERMCOs
print(f"Total count of unique PERMCO: {unique_permco_count}")


In [ ]:
# Check for any negative values in the 'PRC' column
negative_prc = CRSP[CRSP['PRC'] < 0]

# Display the entries with negative 'PRC' values, if any
if not negative_prc.empty:
    print("Negative PRC values found:")
    print(negative_prc)
else:
    print("No negative PRC values found.")


In [ ]:
from IPython.display import display

# Create a DataFrame showing each PERMCO with its associated COMNAM
comnam_tickers = CRSP.groupby('PERMCO')['COMNAM'].apply(set).reset_index()

# Name the columns as requested
comnam_tickers.columns = ['PERMCO', 'Associated Company Name']

# Display the DataFrame in a scrollable output window in Jupyter Notebook
display(comnam_tickers.style.set_table_attributes('style="display:block; height:200px; overflow:auto;"'))


In [ ]:
CRSP.head(5)

In [ ]:
#Summarize of the dataset
def summarize_data(df, name):
    print(f"Summary for {name}:")
    print("Total columns:", df.shape[1])
    print("Total rows:", df.shape[0])
    print("Missing values per column:")
    print(df.isnull().sum())
    print("\n")
summarize_data(CRSP, "CRSP Dataset")

In [ ]:
# Filter out the rows where 'COMNAM' is 'MILLICOM INTL CELLULAR SA' or 'CHARTER COMMUNICATIONS INC'
CRSP = CRSP[~CRSP['COMNAM'].isin(['MILLICOM INTL CELLULAR SA', 'CHARTER COMMUNICATIONS INC'])]

In [ ]:
CRSP['date'] = pd.to_datetime(CRSP['date'])

# Determine the overall date range in the dataset
min_date = CRSP['date'].min()
max_date = CRSP['date'].max()
full_date_range = pd.date_range(start=min_date, end=max_date, freq='D')

# Function to calculate missing dates for a given ticker
def calculate_missing_dates(ticker_data, full_date_range):
    ticker_dates = pd.DatetimeIndex(ticker_data['date'])
    missing_dates = full_date_range.difference(ticker_dates)
    return missing_dates

# Apply the function to each ticker and store the results in a DataFrame
results = {}
for COMNAM in CRSP['COMNAM'].unique():
    ticker_data = CRSP[CRSP['COMNAM'] == COMNAM]
    missing_dates = calculate_missing_dates(ticker_data, full_date_range)
    results[COMNAM] = len(missing_dates)

# Convert results dictionary to DataFrame for better readability and analysis
results_df = pd.DataFrame(list(results.items()), columns=['COMNAM', 'Missing Dates Count'])

# Display the results
print(results_df)


## Drop the missing value, gap, or uncommon value

In [ ]:
# Drop rows with any missing values
CRSP.dropna(inplace=True)

# Drop rows where the 'PRC' column contains negative values
CRSP = CRSP[CRSP['PRC'] > 0]

# Ensure 'RET' contains only numerical values and drop rows with non-numerical values
CRSP = CRSP[pd.to_numeric(CRSP['RET'], errors='coerce').notna()]

In [ ]:
# Print unique PERMCO values and their count
PERMCO_unique = CRSP['PERMCO'].unique()
print("Unique PERMCO values in CRSP")
PERMCO_unique
print("Count of unique COMNAM values:", len(PERMCO_unique))

In [ ]:
#Summarize of the dataset
def summarize_data(df, name):
    print(f"Summary for {name}:")
    print("Total columns:", df.shape[1])
    print("Total rows:", df.shape[0])
    print("Missing values per column:")
    print(df.isnull().sum())
    print("\n")
summarize_data(CRSP, "CRSP Dataset")

In [ ]:
# Group by 'PERMCO' and aggregate the 'date' column to find min and max dates
final_CRSP_processing_check = CRSP.groupby('PERMCO').agg(
    min_date=('date', 'min'),
    max_date=('date', 'max')
).reset_index()

comnam_mapping = CRSP[['PERMCO', 'COMNAM']].drop_duplicates().set_index('PERMCO')
final_CRSP_processing_check['COMNAM'] = final_CRSP_processing_check['PERMCO'].map(comnam_mapping['COMNAM'])

# Show the result
final_CRSP_processing_check


# IQ Key Development Dataset Preprocessing

## Organize Company Name and Ticker of IQKD Dataset

In [ ]:
#Summarize of the dataset
def summarize_data(df, name):
    print(f"Summary for {name}:")
    print("Total columns:", df.shape[1])
    print("Total rows:", df.shape[0])
    print("Missing values per column:")
    print(df.isnull().sum())
    print("\n")
summarize_data(IQKD, "IQ Key Development Dataset")

In [ ]:
# Columns to be removed
columns_to_remove = ['companyid', 'sptodate']

# Drop the specified columns from the IQKD DataFrame
IQKD= IQKD.drop(columns=columns_to_remove)

# Display the modified DataFrame
IQKD.head(3)

In [ ]:
# Display the unique gvkey and companyname
unique_gvkey_companyname = IQKD[['gvkey', 'companyname']].drop_duplicates().reset_index(drop=True)

# Print the unique gvkey and companyname
print(unique_gvkey_companyname)


In [ ]:
# Remove rows with specified company names
companies_to_remove = ['M&A Rumors and Discussions', 'Millicom International Cellular S.A.', 'Charter Communications, Inc.']
IQKD = IQKD[~IQKD['companyname'].isin(companies_to_remove)].reset_index(drop=True)

# Fill NaN gvkey based on similar companyname
# Create a mapping of companyname to gvkey for non-NaN gvkey
company_to_gvkey_map = IQKD.dropna(subset=['gvkey']).set_index('companyname')['gvkey'].to_dict()

# Fill NaN gvkey based on the company_to_gvkey_map
IQKD['gvkey'] = IQKD.apply(
    lambda row: company_to_gvkey_map.get(row['companyname'], row['gvkey']),
    axis=1
)

# Remove duplicate rows based on both gvkey and companyname
IQKD = IQKD.drop_duplicates().reset_index(drop=True)


In [ ]:
# Group by 'PERMCO' and select the first 'gvkey' and 'companyname' for each group
first_values = IQKD.groupby('gvkey').first()

# Map these first values back to the original DataFrame
IQKD['companyname'] = IQKD['gvkey'].map(first_values['companyname'])

# Now CRSP DataFrame should have standardized 'gvkey' and 'companyname' for each 'gvkey'


In [ ]:
#Summarize of the dataset
def summarize_data(df, name):
    print(f"Summary for {name}:")
    print("Total columns:", df.shape[1])
    print("Total rows:", df.shape[0])
    print("Missing values per column:")
    print(df.isnull().sum())
    print("\n")
summarize_data(IQKD, "IQ Key Development Dataset")

In [ ]:
IQKD['announcedate'] = pd.to_datetime(IQKD['announcedate'], errors='coerce')

# Group by the new 'companyname' and analyze the 'announcedate' column
grouped_GVKeycompany = IQKD.groupby('companyname')['announcedate'].agg(['min', 'max', 'count']).reset_index()

# Display the results
print("Unified date range and count of entries for each company before merging:")
grouped_GVKeycompany


In [ ]:
#Summarize of the dataset
def summarize_data(df, name):
    print(f"Summary for {name}:")
    print("Total columns:", df.shape[1])
    print("Total rows:", df.shape[0])
    print("Missing values per column:")
    print(df.isnull().sum())
    print("\n")
summarize_data(IQKD, "IQ Key Development Dataset")

## Removing Redundant Headline on the Same Day for every company

In [ ]:
# Group by 'gvkey' and 'announcedate' and collect headlines
headline_groups = IQKD.groupby(['gvkey', 'announcedate'])['headline'].agg(list)

# Check for redundant headlines within each group
redundant_headlines = headline_groups.apply(lambda headlines: len(headlines) != len(set(headlines)))

# Filter to show only the groups where there are redundant headlines
redundant_headlines = redundant_headlines[redundant_headlines]

# Join this with the original DataFrame to get the rows with redundant headlines
redundant_data = IQKD[IQKD.set_index(['gvkey', 'announcedate']).index.isin(redundant_headlines.index)]

# Display the data
print("Data with redundant headlines:")
redundant_data.head(2)


In [ ]:
# Filter for the specific ticker and date
specific_data = IQKD[(IQKD['gvkey'] == '126136') & (pd.to_datetime(IQKD['announcedate']) == pd.Timestamp('2009-05-20'))]

# Display the relevant rows
print("Entries for Charter Communications:")
print(specific_data)

# To explicitly check for redundant headlines
if specific_data['headline'].duplicated().any():
    print("Redundant headlines found.")
else:
    print("No redundant headlines.")


In [ ]:
# Count rows per Company Name before removing duplicates
count_before = IQKD.groupby('companyname').size()

# Remove rows where 'Company Name', 'announcedate', and 'headline' are identical, keeping the first occurrence
IQKD_cleaned = IQKD.drop_duplicates(subset=['companyname', 'announcedate', 'headline'], keep='first')

# Count rows per Company Name after removing duplicates
count_after = IQKD_cleaned.groupby('companyname').size()

# Combine counts into a DataFrame for side-by-side comparison
comparison_df = pd.DataFrame({
    'Before': count_before,
    'After': count_after
})

# Fill NaN values with 0 if there are any, in case some Company Name disappear completely after cleaning
comparison_df.fillna(0, inplace=True)

# Print the comparison DataFrame
print("Comparison of row counts per Company Name before and after removing Headlines duplicates:")
print(comparison_df)


In [ ]:
# Remove rows where 'gvkey', 'announcedate', and 'headline' are identical except keep the first
IQKD = IQKD.drop_duplicates(subset=['gvkey', 'announcedate', 'headline'], keep='first')

In [ ]:
#Summarize of the dataset
def summarize_data(df, name):
    print(f"Summary for {name}:")
    print("Total columns:", df.shape[1])
    print("Total rows:", df.shape[0])
    print("Missing values per column:")
    print(df.isnull().sum())
    print("\n")
summarize_data(IQKD, "IQ Key Development Dataset")

## Dropping the rows that having missing announcedate

In [ ]:
rows_with_missing_values = IQKD[IQKD.isnull().any(axis=1)]

# Show the first 10 rows that will be dropped
rows_with_missing_values.head(5)

In [ ]:
# Drop rows where 'announcedate' is NaT
IQKD = IQKD[pd.notnull(IQKD['announcedate'])]

In [ ]:
#Summarize of the dataset
def summarize_data(df, name):
    print(f"Summary for {name}:")
    print("Total columns:", df.shape[1])
    print("Total rows:", df.shape[0])
    print("Missing values per column:")
    print(df.isnull().sum())
    print("\n")
summarize_data(IQKD, "IQ Key Development Dataset")

## Put [No_Headline] placeholders on the date where no headlines

In [ ]:
IQKD['announcedate'] = pd.to_datetime(IQKD['announcedate'])

# Sort the DataFrame by 'gvkey' and 'announcedate' to ensure correct date difference calculation
IQKD.sort_values(['gvkey', 'announcedate'], inplace=True)

# Calculate the difference between consecutive dates for each gvkey
IQKD['date_diff'] = IQKD.groupby('gvkey')['announcedate'].diff().dt.days - 1

# Filter to show only the rows where there is a gap (date_diff > 0)
date_gaps = IQKD[IQKD['date_diff'] > 0]

# Optional: summarize the maximum gap for each gvkey
max_gaps = date_gaps.groupby('gvkey')['date_diff'].max()

# Print or view the summary of maximum gaps
print("Maximum date gaps for each company:")
print(max_gaps)


In [ ]:
# Sort the DataFrame by 'gvkey' and 'announcedate' to ensure correct date difference calculation
IQKD.sort_values(['gvkey', 'announcedate'], inplace=True)

# Calculate the difference between consecutive dates for each gvkey
IQKD['date_diff'] = IQKD.groupby('gvkey')['announcedate'].diff().dt.days

# Filter to show only the rows where there is a gap (date_diff > 0)
date_gaps = IQKD[IQKD['date_diff'] > 0]

# Summarize the average gap for each gvkey
average_gaps = date_gaps.groupby('gvkey')['date_diff'].mean().reset_index()

# Retrieve the first company name for each gvkey
first_company_name = IQKD.drop_duplicates(subset='gvkey')[['gvkey', 'companyname']]

# Merge the average gaps with the first company name
average_gaps = average_gaps.merge(first_company_name, on='gvkey')

# Print or view the summary of average gaps with gvkey and companyname
print("Average date gaps for each gvkey and companyname:")
print(average_gaps)


In [ ]:
# Filter to find gaps specifically for Verizon
Verizon_gaps = IQKD[(IQKD['gvkey'] == 2136.0) & (IQKD['date_diff'] > 0)]

# Print the rows with gaps for Google Inc (Alphabet)
print("Rows with gaps for Verizon:")
Verizon_gaps.head(10)


In [ ]:
# Ensure 'announcedate' is in datetime format and sort the DataFrame
IQKD['announcedate'] = pd.to_datetime(IQKD['announcedate'])
IQKD.sort_values(['gvkey', 'announcedate'], inplace=True)

# Function to fill missing dates for each gvkey
def fill_missing_dates(group):
    all_dates = pd.date_range(start=group['announcedate'].min(), end=group['announcedate'].max(), freq='D')
    all_dates_df = pd.DataFrame(all_dates, columns=['announcedate'])
    merged = all_dates_df.merge(group, on='announcedate', how='left')
    if merged.isna().sum().sum() > 0:  # Check if there are any missing values to fill
        merged['gvkey'] = group['gvkey'].iloc[0]
        merged['companyname'] = group['companyname'].iloc[0]
        merged['headline'].fillna('[No_Headline]', inplace=True)
        merged['eventtype'].fillna('[No_Event]', inplace=True)
    return merged

# Apply the function to each gvkey group
IQKD = IQKD.groupby('gvkey').apply(fill_missing_dates).reset_index(drop=True)

# Print or examine updated DataFrame
IQKD.head(10)


In [ ]:
# Filter the DataFrame to show rows where 'headline' is '[No_Headline]'
no_headline_rows = IQKD[IQKD['headline'] == '[No_Headline]']

# Display these rows.
no_headline_rows.head(10)

In [ ]:
IQKD.sort_values(['gvkey', 'announcedate'], inplace=True)

# Filter the DataFrame to show rows where 'headline' is '[No_Headline]'
no_headline_rows = IQKD[IQKD['headline'] == '[No_Headline]']

# Group by 'companyname' and 'announcedate' and count occurrences
no_headline_counts = no_headline_rows.groupby(['companyname', 'announcedate']).size()

# Filter to keep only groups with more than one occurrence
multiple_no_headline = no_headline_counts[no_headline_counts > 1].reset_index()

if multiple_no_headline.empty:
    print("There are no more than 1 [No_Headline] on each company for the same announcedate.")
else:
    # Merge to get detailed rows for these groups
    detailed_no_headline_rows = no_headline_rows.merge(multiple_no_headline, on=['companyname', 'announcedate'])

    # Display these rows
    print("Rows with more than 1 no headline on the same announcedate for each company:")
    print(detailed_no_headline_rows.head(5))


In [ ]:
Verizon_data = IQKD[IQKD['companyname'] == 'Verizon Communications Inc.']

print("Head rows for Verizon:")
Verizon_data.head(10)

## Add 'Situation' from WRDS Cloud

In [ ]:
import wrds
db = wrds.Connection(wrds_username='baguspranata')
db.create_pgpass_file()

In [ ]:
db.close()
db = wrds.Connection(wrds_username='baguspranata')

In [ ]:
table_description = db.describe_table(library='ciq_keydev', table='ciqkeydev')
table_description

In [ ]:
unique_keydevids = IQKD['keydevid'].unique()

In [ ]:
# Convert the list of keydevids to a format suitable for SQL queries
keydevid_list = ','.join([f"'{str(k)}'" for k in unique_keydevids])

# Construct the SQL query
sql_query = f"""
SELECT keydevid, headline, situation, announceddate
FROM ciq_keydev.ciqkeydev
WHERE keydevid IN ({keydevid_list})
"""

# Fetch the data
matched_data = db.raw_sql(sql_query)
matched_data.head()

In [ ]:
# Convert keydevid in IQKD to integer (remove non-numeric entries first if necessary)
IQKD['keydevid'] = pd.to_numeric(IQKD['keydevid'], errors='coerce').dropna().astype(int)

# Convert keydevid in matched_data to integer
matched_data['keydevid'] = pd.to_numeric(matched_data['keydevid'], errors='coerce').dropna().astype(int)

# Merge the situation data from matched_data into IQKD based on keydevid
IQKD = pd.merge(IQKD, matched_data[['keydevid', 'situation']], on='keydevid', how='left')

# Check and fill missing situations after merge
IQKD['situation'] = IQKD['situation'].fillna('[No Situation]')

# Print the resulting DataFrame to check the results
IQKD.head(3)


In [ ]:
# Filter the DataFrame to include only rows where 'situation' is not missing
filtered_IQKD = IQKD[IQKD['situation'].notna()]

# Print the filtered DataFrame
filtered_IQKD.head(2)


In [ ]:
IQKD.sort_values(['gvkey', 'announcedate'], inplace=True)

# Filter the DataFrame to show rows where 'headline' is '[No_Headline]'
no_headline_rows = IQKD[IQKD['headline'] == '[No_Headline]']

# Group by 'companyname' and 'announcedate' and count occurrences
no_headline_counts = no_headline_rows.groupby(['companyname', 'announcedate']).size()

# Filter to keep only groups with more than one occurrence
multiple_no_headline = no_headline_counts[no_headline_counts > 1].reset_index()

if multiple_no_headline.empty:
    print("There are no more than 1 [No_Headline] on each company for the same announcedate.")
else:
    # Merge to get detailed rows for these groups
    detailed_no_headline_rows = no_headline_rows.merge(multiple_no_headline, on=['companyname', 'announcedate'])

    # Display these rows
    print("Rows with more than 1 no headline on the same announcedate for each company:")
    print(detailed_no_headline_rows.head(10))


## Distribution of headlines throughout the time on every ticker

In [ ]:
IQKD['announcedate'] = pd.to_datetime(IQKD['announcedate'])

# Group by 'ticker' and 'announcedate' and count the number of headlines
headline_counts = IQKD.groupby(['companyname', 'announcedate']).size().reset_index(name='headline_count')

# Get unique tickers
unique_tickers = headline_counts['companyname'].unique()

# Calculate the number of rows needed for subplots (2 plots per row)
num_rows = (len(unique_tickers) + 1) // 2

# Create a figure and array of axes with 2 columns
fig, axes = plt.subplots(nrows=num_rows, ncols=2, figsize=(14, num_rows * 4))
axes = axes.flatten()  # Flatten to 1D array for easier iteration

# Plot each ticker's data in its respective subplot
for idx, companyname in enumerate(unique_tickers):
    ax = axes[idx]
    # Filter data for the current companyname
    ticker_data = headline_counts[headline_counts['companyname'] == companyname]

    # Create the line plot on the specified axes
    sns.lineplot(ax=ax, data=ticker_data, x='announcedate', y='headline_count', marker='o', label=companyname)

    # Adding plot details
    ax.set_title(f'Number of Headlines Over Time for {companyname}')
    ax.set_xlabel('Date')
    ax.set_ylabel('Number of Headlines')
    ax.legend()
    ax.grid(True)

# If the number of tickers is odd, hide the last subplot (if unused)
if len(unique_tickers) % 2 != 0:
    axes[-1].set_visible(False)

plt.tight_layout()
plt.show()


# CRSP Weekly Return Computation

In [ ]:
CRSP.head(2)

## Checking the frequency of outliers of PRC and RET on each TICKERs

In [ ]:
# Convert 'RET' and 'PRC' to numeric, coercing errors to NaN
CRSP['RET'] = pd.to_numeric(CRSP['RET'], errors='coerce')
CRSP['PRC'] = pd.to_numeric(CRSP['PRC'], errors='coerce')

# Determine the first company name for each PERMCO
first_comnam = CRSP.sort_values(by='date').groupby('PERMCO')['COMNAM'].first().reset_index()

# Map this first company name back to the main DataFrame using PERMCO
CRSP = CRSP.merge(first_comnam, on='PERMCO', suffixes=('', '_first'))

# Handle NaN values if necessary
# CRSP.dropna(subset=['RET', 'PRC'], inplace=True)  # Option to drop NaNs

# Calculate IQR and bounds within a function to ensure proper grouping
def calculate_outliers_and_proportions(group):
    Q1 = group[['RET', 'PRC']].quantile(0.25)
    Q3 = group[['RET', 'PRC']].quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Count outliers
    ret_outliers = ((group['RET'] < lower_bound['RET']) | (group['RET'] > upper_bound['RET']))
    prc_outliers = ((group['PRC'] < lower_bound['PRC']) | (group['PRC'] > upper_bound['PRC']))

    # Calculate proportions
    ret_outliers_count = ret_outliers.sum()
    prc_outliers_count = prc_outliers.sum()
    ret_proportion = ret_outliers_count / len(group)
    prc_proportion = prc_outliers_count / len(group)

    return pd.Series({'RET_Outliers': ret_outliers_count, 'PRC_Outliers': prc_outliers_count,
                      'RET_Outlier_Proportion': ret_proportion, 'PRC_Outlier_Proportion': prc_proportion})

# Apply function to each group based on the first COMNAM
outlier_counts = CRSP.groupby('COMNAM').apply(calculate_outliers_and_proportions)

# Print the result
outlier_counts


In [ ]:
CRSP.head(2)

## Weekly Return Computation Without PRC & RET Outliers Removal

In [ ]:
import pandas as pd
import numpy as np

# Assuming CRSP is your DataFrame

# Step 1: Convert date to datetime and other data to numeric
CRSP['date'] = pd.to_datetime(CRSP['date'])
CRSP['RET'] = pd.to_numeric(CRSP['RET'], errors='coerce')
CRSP['PRC'] = pd.to_numeric(CRSP['PRC'], errors='coerce')
CRSP['SHROUT'] = pd.to_numeric(CRSP['SHROUT'], errors='coerce')

# Step 2: Filter the data
NonMissing_CRSP = CRSP[(CRSP['PRC'] > 0) & (CRSP['RET'].notna())]

# Step 3: Calculate weekly returns and market cap
# Set index to date for resampling
NonMissing_CRSP.set_index('date', inplace=True)

# Resample to get weekly data
weekly_CRSP = NonMissing_CRSP.groupby(['PERMCO', 'COMNAM']).resample('W-FRI').agg({
    'PRC': 'last',  # Get the last price of the week
    'RET': lambda x: np.exp(np.sum(np.log(1 + x))) - 1,  # Compound weekly return
    'SHROUT': 'last'  # Share outstanding at the end of the week
}).reset_index()

# Calculate market cap
weekly_CRSP['market_cap'] = weekly_CRSP['PRC'] * weekly_CRSP['SHROUT']

# Rename the column to indicate it is the weekly compound return
weekly_CRSP.rename(columns={'RET': 'Weekly Compound Return'}, inplace=True)

# Calculate Return Direction based on price movement compared to the previous week
weekly_CRSP['Past Return Direction'] = np.select(
    [
        weekly_CRSP['PRC'].diff() > 0,
        weekly_CRSP['PRC'].diff() < 0
    ],
    [
        'Up',  # Price went up
        'Down'  # Price went down
    ],
    default='No Change'  # No change in price
)

# Calculate Future Direction by comparing with the next week's price
weekly_CRSP['Future Return Direction'] = np.select(
    [
        weekly_CRSP['PRC'].shift(-1) > weekly_CRSP['PRC'],
        weekly_CRSP['PRC'].shift(-1) < weekly_CRSP['PRC']
    ],
    [
        'Up',  # Price will go up next week
        'Down'  # Price will go down next week
    ],
    default='No Change'  # No change in price next week
)

# Display the head of the modified DataFrame
print(weekly_CRSP.head())


In [ ]:
# Calculate the 'Date From' by subtracting 6 days from 'date' (since 'date' is the end of the week)
weekly_CRSP['Date From'] = weekly_CRSP['date'] - pd.Timedelta(days=6)

# 'Date To' is just the 'date' column, which represents the week ending
weekly_CRSP['Date To'] = weekly_CRSP['date']

# Rearrange columns for better readability, if necessary
weekly_CRSP = weekly_CRSP[['COMNAM','PERMCO','date', 'Date From', 'Date To','market_cap', 'Weekly Compound Return', 'Past Return Direction', 'Future Return Direction']]

In [ ]:
# Provided PERMCO-GVKEY pairs
permco_gvkey_mapping = {
    45483: 160329,
    21645: 9899,
    11204: 24708,
    16779: 122915,
    17322: 126136,
    42769: 147204,
    43613: 3226,
    10303: 16721,
    1908: 5284,
    13136: 30312,
    4922: 11499,
    34913: 28378,
    12746: 30024,
    43145: 147579,
    44625: 149177,
    1367: 4066,
    21866: 13714,
    15708: 65460,
    11358: 9466,
    40213: 9664,
    13758: 60800,
    15436: 64630,
    20782: 4988,
    5230: 10411,
    20997: 6136,
    21280: 7866,
    20587: 3980,
    17205: 125240,
    21826: 14369,
    20288: 2136,
    16665: 122172
}

# Mapping the gvkey to the weekly_CRSP DataFrame
weekly_CRSP['gvkey'] = weekly_CRSP['PERMCO'].map(permco_gvkey_mapping)


In [ ]:
# Display the DataFrame
weekly_CRSP.tail(10)

In [ ]:
# Count unique TICKER values in the weekly_CRSP DataFrame
unique_PERMCO_count = weekly_CRSP['PERMCO'].nunique()
print(f"Number of unique PERMCOs: {unique_PERMCO_count}")


In [ ]:
# Find the latest date
latest_date = weekly_CRSP['Date To'].max()

# Display the latest date
print("The latest date in the 'Date To' column is:", latest_date)


In [ ]:
# Make sure the 'Date To' column is in datetime format
weekly_CRSP['Date To'] = pd.to_datetime(weekly_CRSP['Date To'])

# Set Seaborn's aesthetic parameters to make the plots more visually appealing
sns.set(style="whitegrid")

# Generate a list of years to use as x-axis ticks
years = pd.date_range(start='2005-01-01', end='2024-12-31', freq='YS').year

# Iterate over each group defined by 'COMNAM'
for ticker, group in weekly_CRSP.groupby('COMNAM'):
    fig, ax = plt.subplots(figsize=(12, 2))  # Create a new figure for each ticker
    group = group.sort_values('Date To')  # Ensure data is sorted by date for plotting

    # Use Seaborn's lineplot for better aesthetics; since seaborn is based on matplotlib, it uses the same syntax for ax
    sns.lineplot(data=group, x='Date To', y='Weekly Compound Return', marker='o', linestyle='-', label=f'{ticker} Returns', ax=ax)

    # Set title and labels with enhanced formatting
    ax.set_title(f'Weekly Compound Return for {ticker}', fontsize=12)
    ax.set_xlabel('Date', fontsize=10)
    ax.set_ylabel('Weekly Compound Return', fontsize=10)
    ax.legend()

    # Set x-ticks to display each year
    ax.set_xticks(pd.to_datetime(years, format='%Y'))
    ax.set_xticklabels(years, rotation=45)

    # Show the plot
    plt.show()


In [ ]:
# Setting Seaborn's aesthetic parameters for better visual effects
sns.set(style="whitegrid", palette="pastel")

# Assuming 'weekly_CRSP' is your DataFrame
all_tickers = weekly_CRSP['COMNAM'].unique()  # Get all unique tickers

# Setup the figure size to fit all tickers horizontally with improved vertical dimension
plt.figure(figsize=(0.5 * len(all_tickers), 5))  # Increase the height for better visibility

# Create a seaborn boxplot across a single axis without subplot division
sns.boxplot(x='COMNAM', y='Weekly Compound Return', data=weekly_CRSP, width=0.6)

# Set the plot title and labels with improved formatting
plt.title('Weekly Returns for All COMNAM', fontsize=15)
plt.xlabel('COMNAM', fontsize=14)
plt.ylabel('Weekly Compound Return', fontsize=14)
plt.xticks(rotation=90)  # Rotate ticker labels for better readability if necessary

plt.grid(True)  # Ensure the grid is visible for better readability
plt.show()



# Merging & Split Weekly Return CRSP with IQKD and stored it into desired dataframe structure

In [ ]:
#To avoiding the memory error issue, save the previous IQKD and weekly_CRSP that had been proccessed before.

# Save the IQKD DataFrame to a CSV file
IQKD.to_csv('cleaned_IQKD.csv', index=False)

# Save the weekly_CRSP DataFrame to a CSV file
weekly_CRSP.to_csv('cleaned_weekly_CRSP.csv', index=False)


In [ ]:
# Reload the CRSP and IQ Key Development dataset
weekly_CRSP = pd.read_csv('cleaned_weekly_CRSP.csv')
IQKD = pd.read_csv('cleaned_IQKD.csv')

In [ ]:
# Convert 'announcedate' to datetime format
IQKD['announcedate'] = pd.to_datetime(IQKD['announcedate'])

# Filter out rows where headline is '[No_Headline]'
filtered_IQKD = IQKD[IQKD['headline'] != '[No_Headline]']

# Extract year and month from 'announcedate'
filtered_IQKD['year'] = filtered_IQKD['announcedate'].dt.year
filtered_IQKD['month'] = filtered_IQKD['announcedate'].dt.month_name()

# Set the style of the plots
sns.set(style="whitegrid")

# Define the color
viridis_blue = '#1f77b4'  # Hex code for a blue color in the Viridis palette

# Plot 1: Total Counts of headlines based on year
plt.figure(figsize=(10, 6))
yearly_counts = filtered_IQKD['year'].value_counts().sort_index()
sns.barplot(x=yearly_counts.index, y=yearly_counts.values, color=viridis_blue)
plt.title('Total Counts of Headlines Based on Year')
plt.xlabel('Year')
plt.ylabel('Total Counts')
plt.xticks(rotation=0)
plt.grid(axis='y', linestyle='--')
plt.show()

# Plot 2: Total Counts of headlines based on month
plt.figure(figsize=(12, 6))
monthly_counts = filtered_IQKD['month'].value_counts().reindex([
    'January', 'February', 'March', 'April', 'May', 'June', 'July',
    'August', 'September', 'October', 'November', 'December'
])
sns.barplot(x=monthly_counts.index, y=monthly_counts.values, color=viridis_blue)
plt.title('Total Counts of Headlines Based on Month')
plt.xlabel('Month')
plt.ylabel('Total Counts')
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--')
plt.show()


In [ ]:
# Ensure that the 'Date From', 'Date To' in weekly_CRSP and 'announcedate' in IQKD are datetime
weekly_CRSP['Date From'] = pd.to_datetime(weekly_CRSP['Date From'], errors='coerce')
weekly_CRSP['Date To'] = pd.to_datetime(weekly_CRSP['Date To'], errors='coerce')
IQKD['announcedate'] = pd.to_datetime(IQKD['announcedate'], errors='coerce')

# Function to split DataFrame into in-sample and out-sample based on a split date
def split_dataframes(df, date_column, split_date):
    in_sample = df[df[date_column] < split_date]
    out_sample = df[df[date_column] >= split_date]
    return in_sample, out_sample

# Define the split date
split_date = pd.to_datetime('2016-01-01')

# Split weekly_CRSP DataFrame
InSample_weekly_CRSP, OutSample_weekly_CRSP = split_dataframes(weekly_CRSP, 'Date From', split_date)

# Split IQKD DataFrame
InSample_IQKD, OutSample_IQKD = split_dataframes(IQKD, 'announcedate', split_date)

# Display the resulting DataFrames
print("InSample_weekly_CRSP:")
print(InSample_weekly_CRSP.head(1))

print("\nOutSample_weekly_CRSP:")
print(OutSample_weekly_CRSP.head(1))

print("\nInSample_IQKD:")
print(InSample_IQKD.head(1))

print("\nOutSample_IQKD:")
print(OutSample_IQKD.head(1))


In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)  # Be cautious with this on large DataFrames


In [ ]:
weekly_CRSP.tail(2)

In [ ]:
IQKD.tail(10)


In [ ]:
# Merge InSample DataFrames
insample_df = pd.merge(InSample_weekly_CRSP, InSample_IQKD, left_on='gvkey', right_on='gvkey')
insample_df = insample_df[(insample_df['announcedate'] >= insample_df['Date From']) & (insample_df['announcedate'] <= insample_df['Date To'])]
insample_df = insample_df[['companyname', 'Date From', 'Date To', 'date', 'Weekly Compound Return', 'Past Return Direction','Future Return Direction','market_cap', 'headline', 'eventtype']]

# Print or return the resulting DataFrames
print("Merged In-Sample DataFrame:")
insample_df.head(5)

In [ ]:
# Filter the DataFrame to show rows where 'headline' is '[No_Headline]'
no_headline_rows = insample_df[insample_df['headline'] == '[No_Headline]']

# Group by 'companyname' and 'announcedate' and count occurrences
no_headline_counts = no_headline_rows.groupby(['companyname', 'date']).size()

# Filter to keep only groups with more than one occurrence
multiple_no_headline = no_headline_counts[no_headline_counts > 1].reset_index()

# Merge to get detailed rows for these groups
detailed_no_headline_rows = no_headline_rows.merge(multiple_no_headline, on=['companyname', 'date'])

# Display these rows
print("INSAMPLE Rows with more than 1 no headline on the same date for each company:")
detailed_no_headline_rows.head(5)


In [ ]:
# Filter the DataFrame to show rows where 'headline' is '[No_Headline]'
no_headline_rows = insample_df[insample_df['headline'] == '[No_Headline]']

# Group by 'companyname' and 'date' and count occurrences
no_headline_counts = no_headline_rows.groupby(['companyname', 'date']).size()

# Filter to keep only groups with more than one occurrence
multiple_no_headline = no_headline_counts[no_headline_counts > 1].reset_index()

# Merge to get detailed rows for these groups
detailed_no_headline_rows = no_headline_rows.merge(multiple_no_headline, on=['companyname', 'date'])

# Display these rows
print("INSAMPLE Rows with more than 1 no headline on the same date for each company:")
print(detailed_no_headline_rows.head(5))

# Remove duplicates keeping only the first occurrence
insample_df_cleaned = insample_df.drop_duplicates(subset=['companyname', 'date', 'headline'], keep='first')

# Save the cleaned DataFrame back into insample_df
insample_df = insample_df_cleaned

# Verify the cleaning process
no_headline_cleaned = insample_df[insample_df['headline'] == '[No_Headline]']
multiple_no_headline_cleaned = no_headline_cleaned.groupby(['companyname', 'date']).size()

# Filter to keep only groups with more than one occurrence after cleaning
multiple_no_headline_cleaned = multiple_no_headline_cleaned[multiple_no_headline_cleaned > 1].reset_index()

# Check if there are any duplicates left
if not multiple_no_headline_cleaned.empty:
    print("There are still rows with more than 1 no headline on the same date for each company after cleaning:")
    print(multiple_no_headline_cleaned.head(5))
else:
    print("No more duplicate 'No_Headline' rows found for the same date and company after cleaning.")

# Display cleaned DataFrame
print("Cleaned INSAMPLE DataFrame:")
print(insample_df.head(5))


In [ ]:
insample_df.head(5)

In [ ]:
# Merge OutSample DataFrames
outsample_df = pd.merge(OutSample_weekly_CRSP, OutSample_IQKD, left_on='gvkey', right_on='gvkey')
outsample_df = outsample_df[(outsample_df['announcedate'] >= outsample_df['Date From']) & (outsample_df['announcedate'] <= outsample_df['Date To'])]
outsample_df = outsample_df[['companyname', 'Date From', 'Date To', 'date', 'Weekly Compound Return', 'Past Return Direction','Future Return Direction','market_cap', 'headline', 'eventtype']]

print("\nMerged Out-Sample DataFrame:")
outsample_df.head(5)

In [ ]:
# Filter the DataFrame to show rows where 'headline' is '[No_Headline]'
no_headline_rows = outsample_df[outsample_df['headline'] == '[No_Headline]']

# Group by 'companyname' and 'announcedate' and count occurrences
no_headline_counts = no_headline_rows.groupby(['companyname', 'date']).size()

# Filter to keep only groups with more than one occurrence
multiple_no_headline = no_headline_counts[no_headline_counts > 1].reset_index()

# Merge to get detailed rows for these groups
detailed_no_headline_rows = no_headline_rows.merge(multiple_no_headline, on=['companyname', 'date'])

# Display these rows
print("OUTSAMPLE Rows with more than 1 no headline on the same date for each company:")
detailed_no_headline_rows.head(5)


In [ ]:
# Filter the DataFrame to show rows where 'headline' is '[No_Headline]'
no_headline_rows = outsample_df[outsample_df['headline'] == '[No_Headline]']

# Group by 'companyname' and 'date' and count occurrences
no_headline_counts = no_headline_rows.groupby(['companyname', 'date']).size()

# Filter to keep only groups with more than one occurrence
multiple_no_headline = no_headline_counts[no_headline_counts > 1].reset_index()

# Merge to get detailed rows for these groups
detailed_no_headline_rows = no_headline_rows.merge(multiple_no_headline, on=['companyname', 'date'])

# Display these rows
print("OUTSAMPLE Rows with more than 1 no headline on the same date for each company:")
print(detailed_no_headline_rows.head(5))

# Remove duplicates keeping only the first occurrence
outsample_df_cleaned = outsample_df.drop_duplicates(subset=['companyname', 'date', 'headline'], keep='first')

# Save the cleaned DataFrame back into outsample_df
outsample_df = outsample_df_cleaned

# Verify the cleaning process
no_headline_cleaned = outsample_df[outsample_df['headline'] == '[No_Headline]']
multiple_no_headline_cleaned = no_headline_cleaned.groupby(['companyname', 'date']).size()

# Filter to keep only groups with more than one occurrence after cleaning
multiple_no_headline_cleaned = multiple_no_headline_cleaned[multiple_no_headline_cleaned > 1].reset_index()

# Check if there are any duplicates left
if not multiple_no_headline_cleaned.empty:
    print("There are still rows with more than 1 no headline on the same date for each company after cleaning:")
    print(multiple_no_headline_cleaned.head(5))
else:
    print("No more duplicate 'No_Headline' rows found for the same date and company after cleaning.")

# Display cleaned DataFrame
print("Cleaned OUTSAMPLE DataFrame:")
print(outsample_df.head(5))


In [ ]:
# Filter the DataFrame to show rows where 'headline' is '[No_Headline]'
no_headline_rows = outsample_df[outsample_df['headline'] == '[No_Headline]']

# Group by 'companyname' and 'announcedate' and count occurrences
no_headline_counts = no_headline_rows.groupby(['companyname', 'date']).size()

# Filter to keep only groups with more than one occurrence
multiple_no_headline = no_headline_counts[no_headline_counts > 1].reset_index()

# Merge to get detailed rows for these groups
detailed_no_headline_rows = no_headline_rows.merge(multiple_no_headline, on=['companyname', 'date'])

# Display these rows
print("OUTSAMPLE Rows with more than 1 no headline on the same date for each company:")
detailed_no_headline_rows.head(5)


In [ ]:
# Remove the column '0' from insample_df if it exists
if '0' in insample_df.columns:
    insample_df = insample_df.drop(columns=['0'])

# Remove the column '0' from outsample_df if it exists
if '0' in outsample_df.columns:
    outsample_df = outsample_df.drop(columns=['0'])


In [ ]:
# Count the unique company names in insample_df
unique_companynames_insample = insample_df['companyname'].nunique()

# Count the unique company names in outsample_df
unique_companynames_outsample = outsample_df['companyname'].nunique()

# Print the results
print(f"Unique company names in insample_df: {unique_companynames_insample}")
print(f"Unique company names in outsample_df: {unique_companynames_outsample}")


In [ ]:
# Get unique company names from each DataFrame
unique_companynames_insample = insample_df['companyname'].unique()
unique_companynames_outsample = outsample_df['companyname'].unique()

# Combine the unique company names and get the unique values across both DataFrames
all_unique_companynames = pd.unique(pd.concat([pd.Series(unique_companynames_insample), pd.Series(unique_companynames_outsample)]))

# Sort the unique company names
sorted_unique_companynames = sorted(all_unique_companynames)

# Print the sorted unique company names
print("Sorted unique company names from both DataFrames:")
for company in sorted_unique_companynames:
    print(company)


In [ ]:
# List of companies to remove
companies_to_remove = ['Gray Television, Inc.', 'Scholastic Corporation', 'RADCOM Ltd.', 'ATN International, Inc.']

# Check if 'companyname' column exists in both DataFrames
if 'companyname' in insample_df.columns and 'companyname' in outsample_df.columns:
    # Filter insample_df
    insample_df = insample_df[~insample_df['companyname'].isin(companies_to_remove)]

    # Filter outsample_df
    outsample_df = outsample_df[~outsample_df['companyname'].isin(companies_to_remove)]


In [ ]:
# Count the unique company names in insample_df
unique_companynames_insample = insample_df['companyname'].nunique()

# Count the unique company names in outsample_df
unique_companynames_outsample = outsample_df['companyname'].nunique()

# Print the results
print(f"Unique company names in insample_df: {unique_companynames_insample}")
print(f"Unique company names in outsample_df: {unique_companynames_outsample}")


In [ ]:
# Get unique company names from each DataFrame
unique_companynames_insample = insample_df['companyname'].unique()
unique_companynames_outsample = outsample_df['companyname'].unique()

# Combine the unique company names and get the unique values across both DataFrames
all_unique_companynames = pd.unique(pd.concat([pd.Series(unique_companynames_insample), pd.Series(unique_companynames_outsample)]))

# Sort the unique company names
sorted_unique_companynames = sorted(all_unique_companynames)

# Print the sorted unique company names
print("Sorted unique company names from both DataFrames:")
for company in sorted_unique_companynames:
    print(company)


In [ ]:
# Save the Insample DataFrame to a CSV file
insample_df.to_csv('insample_df.csv', index=False)

# Save the Outsample DataFrame to a CSV file
outsample_df.to_csv('outsample_df.csv', index=False)

# Tokenize and Word Cloud

In [ ]:
!pip install nltk wordcloud matplotlib


In [ ]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
from wordcloud import WordCloud
from collections import Counter

# Download stopwords from NLTK
import nltk
nltk.download('punkt')
nltk.download('stopwords')

In [ ]:
# Reload the insample and outsample datasets
insample_df = pd.read_csv('insample_df.csv')
outsample_df = pd.read_csv('outsample_df.csv')

## Visualization

In [ ]:
insample_df.head(10)

In [ ]:
# Function to remove duplicate Date From and Date To for each company
def remove_duplicates(df):
    return df.drop_duplicates(subset=['companyname', 'Date From', 'Date To'])

# Remove duplicates
insample_df = remove_duplicates(insample_df)

def plot_weekly_return(df, title):
    # Convert 'Date From' and 'Date To' to datetime format
    df['Date From'] = pd.to_datetime(df['Date From'])
    df['Date To'] = pd.to_datetime(df['Date To'])

    unique_companies = sorted(df['companyname'].unique())  # Sort companies alphabetically

    # Calculate ideal plot dimensions based on number of companies
    num_cols = 5
    num_rows = (len(unique_companies) + num_cols - 1) // num_cols

    fig, axes = plt.subplots(num_rows, num_cols, figsize=(25, 25), sharey=True)  # Adjusted size
    fig.suptitle(title, fontsize=20, fontweight='bold', color='navy')

    for i, company in enumerate(unique_companies):
        row, col = divmod(i, num_cols)
        ax = axes[row, col]

        company_data = df[df['companyname'] == company].sort_values(by='Date From')

        # Use line plot for better visualization of trends
        ax.plot(company_data['Date From'], company_data['Weekly Compound Return'],
                linewidth=1.5, alpha=0.7, color='blue')

        ax.set_title(company, fontsize=12, fontweight='bold')  # Smaller title
        ax.set_xlabel('Date', fontsize=10, fontweight='bold')    # Smaller label
        ax.set_ylabel('Weekly Return', fontsize=10, fontweight='bold')
        ax.grid(True, linestyle='--', linewidth=0.5)

        # Set vertical scale from -0.5 to 0.75
        ax.set_ylim(-0.5, 0.75)

        # Dynamically adjust x-axis ticks based on date range
        date_range = company_data['Date From'].max() - company_data['Date From'].min()
        if date_range > pd.Timedelta(days=365*2):
            ax.xaxis.set_major_locator(mdates.YearLocator())
            ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
        elif date_range > pd.Timedelta(days=365):
            ax.xaxis.set_major_locator(mdates.MonthLocator(bymonth=[1, 4, 7, 10]))
            ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
        else:
            ax.xaxis.set_major_locator(mdates.MonthLocator())
            ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))

        ax.tick_params(axis='x', rotation=45, labelsize=10)
        ax.tick_params(axis='y', labelsize=10)

    # Remove any unused subplots
    for i in range(len(unique_companies), num_rows * num_cols):
        fig.delaxes(axes.flatten()[i])

    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()

plot_weekly_return(insample_df, 'In-sample Weekly Compound Return')


In [ ]:
# Function to remove duplicate Date From and Date To for each company
def remove_duplicates(df):
    return df.drop_duplicates(subset=['companyname', 'Date From', 'Date To'])

# Remove duplicates
outsample_df = remove_duplicates(outsample_df)

def plot_weekly_return(df, title):
    # Convert 'Date From' and 'Date To' to datetime format
    df['Date From'] = pd.to_datetime(df['Date From'])
    df['Date To'] = pd.to_datetime(df['Date To'])

    unique_companies = sorted(df['companyname'].unique())  # Sort companies alphabetically

    # Calculate ideal plot dimensions based on number of companies
    num_cols = 5
    num_rows = (len(unique_companies) + num_cols - 1) // num_cols

    fig, axes = plt.subplots(num_rows, num_cols, figsize=(25, 25), sharey=True)  # Adjusted size
    fig.suptitle(title, fontsize=20, fontweight='bold', color='navy')

    for i, company in enumerate(unique_companies):
        row, col = divmod(i, num_cols)
        ax = axes[row, col]

        company_data = df[df['companyname'] == company].sort_values(by='Date From')

        # Use line plot for better visualization of trends
        ax.plot(company_data['Date From'], company_data['Weekly Compound Return'],
                linewidth=1.5, alpha=0.7, color='blue')

        ax.set_title(company, fontsize=12, fontweight='bold')  # Smaller title
        ax.set_xlabel('Date', fontsize=10, fontweight='bold')    # Smaller label
        ax.set_ylabel('Weekly Return', fontsize=10, fontweight='bold')
        ax.grid(True, linestyle='--', linewidth=0.5)

        # Set vertical scale from -0.5 to 0.75
        ax.set_ylim(-0.5, 0.75)

        # Dynamically adjust x-axis ticks based on date range
        date_range = company_data['Date From'].max() - company_data['Date From'].min()
        if date_range > pd.Timedelta(days=365*2):
            ax.xaxis.set_major_locator(mdates.YearLocator())
            ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
        elif date_range > pd.Timedelta(days=365):
            ax.xaxis.set_major_locator(mdates.MonthLocator(bymonth=[1, 4, 7, 10]))
            ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
        else:
            ax.xaxis.set_major_locator(mdates.MonthLocator())
            ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))

        ax.tick_params(axis='x', rotation=45, labelsize=10)
        ax.tick_params(axis='y', labelsize=10)

    # Remove any unused subplots
    for i in range(len(unique_companies), num_rows * num_cols):
        fig.delaxes(axes.flatten()[i])

    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()

plot_weekly_return(outsample_df, 'Out-of-sample Weekly Compound Return')


In [ ]:
import matplotlib.dates as mdates
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Function to remove duplicate Date From and Date To for each company
def remove_duplicates(df):
    return df.drop_duplicates(subset=['companyname', 'Date From', 'Date To'])

# Remove duplicates from insample_df and outsample_df
insample_df = remove_duplicates(insample_df)
outsample_df = remove_duplicates(outsample_df)

# Set the plot style
sns.set(style="whitegrid")

def plot_weekly_return_for_company(df, company_name, title):
    # Convert 'Date From' and 'Date To' to datetime format
    df['Date From'] = pd.to_datetime(df['Date From'], format='%Y-%m-%d', errors='coerce')
    df['Date To'] = pd.to_datetime(df['Date To'], format='%Y-%m-%d', errors='coerce')

    # Drop rows where 'Date From' conversion failed
    df = df.dropna(subset=['Date From'])

    # Ensure the company_data is sorted by 'Date From'
    company_data = df[df['companyname'] == company_name].sort_values(by='Date From')

    if company_data.empty:
        print(f"No data available for {company_name} in {title}")
        return

    fig, ax = plt.subplots(figsize=(12, 6))

    # Use line plot for better visualization of trends
    ax.plot(company_data['Date From'], company_data['Weekly Compound Return'],
            linewidth=1.5, alpha=0.7, color='blue', label='Weekly Returns')

    ax.set_title(f"{company_name} - {title}", fontsize=20, fontweight='bold', color='navy')
    ax.set_xlabel('Date', fontsize=14, fontweight='bold')
    ax.set_ylabel('Weekly Return', fontsize=14, fontweight='bold')
    ax.grid(True, linestyle='--', linewidth=0.5)

    # Set the x-axis to show every year
    ax.xaxis.set_major_locator(mdates.YearLocator())
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))

    # Rotate the x-axis labels for better readability
    plt.xticks(rotation=45, fontsize=12)
    plt.yticks(fontsize=12)

    # Remove the top and right spines
    sns.despine()

    plt.tight_layout()
    plt.show()

# Plot for AT&T Inc. in insample and outsample datasets
plot_weekly_return_for_company(insample_df, 'AT&T Inc.', 'In sample Weekly Return')
plot_weekly_return_for_company(outsample_df, 'AT&T Inc.', 'Out of sample Weekly Return')


In [ ]:
# Filtering out rows where headline is '[No_Headline]'
insample_df_filtered = insample_df[insample_df['headline'] != '[No_Headline]']
outsample_df_filtered = outsample_df[outsample_df['headline'] != '[No_Headline]']

# Grouping by companyname and counting headlines
insample_counts = insample_df_filtered.groupby('companyname')['headline'].count().reset_index()
insample_counts.columns = ['companyname', 'insample_count']

outsample_counts = outsample_df_filtered.groupby('companyname')['headline'].count().reset_index()
outsample_counts.columns = ['companyname', 'out-of-sample_count']

# Merging counts
counts_df = pd.merge(insample_counts, outsample_counts, on='companyname', how='outer').fillna(0)

# Calculating the average counts
average_count = counts_df[['insample_count', 'out-of-sample_count']].mean().mean()

# Melt the dataframe for easier plotting with Seaborn
counts_df_melted = counts_df.melt(id_vars='companyname', value_vars=['insample_count', 'out-of-sample_count'],
                                  var_name='Type', value_name='Count')

# Plotting with Seaborn
plt.figure(figsize=(14, 8))
sns.set(style='whitegrid')

# Create the bar plot
bar_plot = sns.barplot(data=counts_df_melted, x='companyname', y='Count', hue='Type', palette='viridis')

# Adding the average line
plt.axhline(y=average_count, color='r', linestyle='--', linewidth=2, label='Average Count')

# Customizing the plot
plt.title('Headline Count per Company in In-sample and Out-of-sample Data')
plt.xlabel('Company')
plt.ylabel('Headline Count')
plt.xticks(rotation=90)
plt.legend(title='Type')

# Display the plot
plt.show()


## Tokenizing, Top Tokens overall and aggregated on each stocks, and Word Cloud


In [ ]:
# Function to preprocess and tokenize text, removing tokens that contain any part of the company name and specific tokens
def tokenize(text, company_name):
    company_parts = set(company_name.lower().split())
    additional_tokens_to_remove = {"'s", "google"}

    # Generate a set of years from 2004 to 2023
    years_to_remove = {str(year) for year in range(2004, 2024)}

    tokens_to_remove = company_parts.union(additional_tokens_to_remove, years_to_remove)

    tokens = word_tokenize(text.lower())  # tokenize and convert to lower case
    tokens = [word for word in tokens if word not in stopwords.words('english') and word not in string.punctuation]
    tokens = [word for word in tokens if word not in tokens_to_remove]  # remove tokens containing any part of the company name and specific tokens
    return tokens

# Filter out '[No_Headline]' before tokenizing
insample_filtered = insample_df[insample_df['headline'] != '[No_Headline]']
outsample_filtered = outsample_df[outsample_df['headline'] != '[No_Headline]']

# Tokenize headlines, passing the company name to the tokenize function
insample_filtered['tokens'] = insample_filtered.apply(lambda row: tokenize(row['headline'], row['companyname']), axis=1)
outsample_filtered['tokens'] = outsample_filtered.apply(lambda row: tokenize(row['headline'], row['companyname']), axis=1)

# Function to aggregate tokens by stock and globally
def aggregate_tokens(data):
    all_tokens = []
    token_counts_per_stock = {}

    # Iterate over each row and update token counts per stock and globally
    for _, row in data.iterrows():
        tokens = row['tokens']
        stock = row['companyname']
        all_tokens.extend(tokens)

        if stock not in token_counts_per_stock:
            token_counts_per_stock[stock] = Counter()
        token_counts_per_stock[stock].update(tokens)

    # Global token counts
    global_token_counts = Counter(all_tokens)

    return token_counts_per_stock, global_token_counts

# Aggregate tokens for insample and outsample data
insample_token_counts_per_stock, insample_global_token_counts = aggregate_tokens(insample_filtered)
outsample_token_counts_per_stock, outsample_global_token_counts = aggregate_tokens(outsample_filtered)

# Function to display top tokens in a table
def display_top_tokens(token_counts, num_top_tokens=10):
    return pd.DataFrame({
        'Token': [token for token, _ in token_counts.most_common(num_top_tokens)],
        'Count': [count for _, count in token_counts.most_common(num_top_tokens)]
    })


In [ ]:
# Example usage: Display top 10 tokens for a specific stock and globally
print("In-sample Global Top Tokens:")
print(display_top_tokens(insample_global_token_counts))
print("\nOut-sample Global Top Tokens:")
print(display_top_tokens(outsample_global_token_counts))

# Example for a specific stock, say 'AT&T Inc.'
print("\nIn-sample AT&T Inc. Top Tokens:")
if 'AT&T Inc.' in insample_token_counts_per_stock:
    print(display_top_tokens(insample_token_counts_per_stock['AT&T Inc.']))
else:
    print("No data for AT&T Inc.")

# Example for a specific stock, say 'AT&T Inc.'
print("\nOut-sample AT&T Inc. Top Tokens:")
if 'AT&T Inc.' in outsample_token_counts_per_stock:
    print(display_top_tokens(outsample_token_counts_per_stock['AT&T Inc.']))
else:
    print("No data for AT&T Inc.")

In [ ]:
# Function to plot top tokens as a bar chart with improved aesthetics
def plot_top_tokens(token_counts, title, num_top_tokens=10):
    tokens, counts = zip(*token_counts.most_common(num_top_tokens))
    plt.figure(figsize=(12, 6))

    # Use seaborn for better aesthetics
    sns.barplot(x=list(tokens), y=list(counts), palette='viridis')

    # Add value annotations on bars
    for i, count in enumerate(counts):
        plt.text(i, count + max(counts) * 0.01, str(count), ha='center', va='bottom')

    plt.xlabel('Tokens', fontsize=14)
    plt.ylabel('Counts', fontsize=14)
    plt.title(title, fontsize=16)
    plt.xticks(rotation=45, ha='right')
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()

# Plotting in-sample and out-sample global top tokens
plot_top_tokens(insample_global_token_counts, 'In-sample Global Top-10 Tokens')
plot_top_tokens(outsample_global_token_counts, 'Out-of-sample Global Top-10 Tokens')

# Plotting for a specific stock, say 'AT&T Inc.'
if 'AT&T Inc.' in outsample_token_counts_per_stock:
    plot_top_tokens(outsample_token_counts_per_stock['AT&T Inc.'], 'Out-of-sample AT&T Inc. Top-10 Tokens')
else:
    print("No data for AT&T Inc.")

# Plotting for a specific stock, say 'AT&T Inc.'
if 'AT&T Inc.' in insample_token_counts_per_stock:
    plot_top_tokens(insample_token_counts_per_stock['AT&T Inc.'], 'In-of-sample AT&T Inc. Top-10 Tokens')
else:
    print("No data for AT&T Inc.")


In [ ]:
import math

# Function to plot word clouds with improved aesthetics
def plot_word_cloud(dataframe, title, cols=5):
    # Group by companyname and concatenate tokens
    grouped_tokens = dataframe.groupby('companyname')['tokens'].sum()
    num_plots = len(grouped_tokens)
    rows = math.ceil(num_plots / cols)

    fig, axes = plt.subplots(rows, cols, figsize=(20, rows * 4))
    fig.suptitle(title, fontsize=20)

    for idx, (companyname, tokens) in enumerate(grouped_tokens.items()):
        row = idx // cols
        col = idx % cols
        ax = axes[row, col] if rows > 1 else axes[col]

        wordcloud = WordCloud(width=800, height=400,
                              max_words=100,
                              background_color='white',
                              colormap='viridis',
                              collocations=False).generate(" ".join(tokens))

        ax.imshow(wordcloud, interpolation='bilinear')
        ax.set_title(companyname, fontsize=12)
        ax.axis("off")

    # Hide any empty subplots
    for idx in range(num_plots, rows * cols):
        row = idx // cols
        col = idx % cols
        ax = axes[row, col] if rows > 1 else axes[col]
        ax.axis("off")

    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

# Plot word clouds for each companyname in insample and outsample datasets
plot_word_cloud(insample_filtered, 'In-sample Word Clouds')
plot_word_cloud(outsample_filtered, 'Out-sample Word Clouds')


In [ ]:
# Function to plot word cloud for a specific company
def plot_word_cloud(dataframe, companyname, title):
    # Filter dataframe for the specific company
    company_data = dataframe[dataframe['companyname'] == companyname]

    # Concatenate tokens for the specific company
    tokens = company_data['tokens'].sum()

    # Generate word cloud with custom settings
    wordcloud = WordCloud(width=800, height=400,
                          max_words=100,  # limit number of words
                          background_color='white',  # set background color
                          colormap='viridis',  # use seaborn colormap
                          font_path=None,  # you can set a specific font path here
                          collocations=False).generate(" ".join(tokens))

    # Plot word cloud
    plt.figure(figsize=(12, 6))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.title(f'Word Cloud for {companyname} - {title}', fontsize=16)
    plt.axis("off")
    plt.tight_layout(pad=0)
    plt.show()

# Plot word cloud for 'AT&T Inc.' in insample and outsample datasets
plot_word_cloud(insample_filtered, 'AT&T Inc.', 'Insample')
plot_word_cloud(outsample_filtered, 'AT&T Inc.', 'Outsample')
